### 동네예보조회
동네예보 정보를 조회하기 위해 발표일자, 발표시각, 예보지점 X좌표, 예보지점 Y 좌표의 조회 조건으로 발표일자, 발표시각, 자료구분문자, 예보 값, 예보일자, 예보시각, 예보지점 X 좌표, 예보지점 Y 좌표의 정보를 조회하는 기능

In [1]:
import datetime
import requests 
import json 
import pandas as pd 

In [2]:
# 동네예보조회 날씨 API 호출하기 위한 주요 값 정의 
CallBackURL = "http://apis.data.go.kr/1360000/VilageFcstInfoService/getVilageFcst?"
# service_key = "<YOUR SERVICE KEY>"
service_key = "gC8RUN3kxgc5v1ZNPvEYW%2FspE4YwHYOn7VWWlIQGfGw2fNVEBCrzvE8cHUFEZpxk6jHmkvwHK9RuL2EjzNz4WQ%3D%3D"

# 첨부된 "기상청18_동네예보 조회서비스_오픈API활용가이드_격자_위경도.xlsx" 에서 참고한 위경도
# 서울시 마포구 연남동 : 
nx = "59"
ny = "127"

# 날씨 API 호출 결과를 정리하기위한 설정 
w_items = {"POP":["강수확률", "%"], 
           "PTY":["강수형태", ""], 
           "R06":["6시간 강수량", "mm"], 
           "REH":["습도", "%"], 
           "S06":["6시간 신적설", "cm"], 
           "SKY":["하늘상태", ""], 
           "T3H":["3시간 기온", "°C"], 
           "TMN":["아침 최저 기온", "°C"], 
           "TMX":["낮 최고 기온", "°C"], 
           "UUU":["풍속(동서성분)","m/s"], 
           "VVV":["풍속(남북성분)","m/s"], 
           "WAV":["파고", "m"], 
           "VEC":["풍향", ""], 
           "WSD":["풍속", "m/s"]}
wind_dict = {0:"N",1:"NNE",2:"NE",3:"ENE",4:"E",5:"ESE",6:"SE",7:"SSE",8:"S",9:"SSW",10:"SW",11:"WSW",12:"W",13:"WNW",14:"NW",15:"NNW",16:"N"}
rain_dict = {0:"없음", 1:"비", 2:"진눈깨비", 3:"눈"}
 


In [3]:
# 동네예보조회 날씨 API 처리를 위한 함수 정의
# 가장 최근의 기상청 예보 시간(02:10 이후 3시간 단위) get
def get_rpt_time():
    base_time = datetime.datetime.now() - datetime.timedelta(hours=2) - datetime.timedelta(minutes=10)
    hour = int(base_time.strftime("%H"))
    rehour = str(hour//3*3 + 2).zfill(2)
    return [base_time.strftime("%Y%m%d"), rehour + "00"]
 
# 풍향 계산용
def get_wind_type(i):
    return wind_dict[(i + 22.5 * 0.5)//22.5]
 
# 하늘 상태 표시용
def get_sky_status(i):
    if i >= 0 and i <= 2: return "맑음"
    elif i >= 3 and i <= 5: return "구름조금"
    elif i >= 6 and i <= 8: return "구름많음"
    elif i >= 9 and i <= 10: return "흐림"
    
def get_real_value(c, s):
    if str(c) == "VEC":
        return get_wind_type(int(str(s)))
    elif str(c) == "SKY":
        return get_sky_status(int(str(s)))
    elif str(c) == "PTY":
        return rain_dict[int(str(s))]
    return str(s)
 

In [4]:
# 동네예보조회 날씨 API 호출 처리를 위한 입력값 정의(Payload)
date = get_rpt_time()

#     "numOfRows=10" + "&" +\
#     "pageNo=1" + "&" +\

payload = "serviceKey=" + service_key + "&" +\
    "dataType=JSON" + "&" +\
    "numOfRows=10" + "&" +\
    "pageNo=1" + "&" +\
    "base_date=" + date[0] + "&" +\
    "base_time=" + date[1] + "&" +\
    "nx=" + nx + "&" +\
    "ny=" + ny

payload

'serviceKey=gC8RUN3kxgc5v1ZNPvEYW%2FspE4YwHYOn7VWWlIQGfGw2fNVEBCrzvE8cHUFEZpxk6jHmkvwHK9RuL2EjzNz4WQ%3D%3D&dataType=JSON&numOfRows=10&pageNo=1&base_date=20210420&base_time=1400&nx=59&ny=127'

In [5]:
# 동네예보조회 날씨 API 호출결과  처리 - 정상적 호출결과인 Json형태 여부 체크
response = requests.get(CallBackURL + payload)

if 'json' in response.headers.get('Content-Type'):
    data = json.loads(response.text)
else:
    print('Response content is not in JSON format.')
    data = 'spam'

data

{'response': {'header': {'resultCode': '00', 'resultMsg': 'NORMAL_SERVICE'},
  'body': {'dataType': 'JSON',
   'items': {'item': [{'baseDate': '20210420',
      'baseTime': '1400',
      'category': 'POP',
      'fcstDate': '20210420',
      'fcstTime': '1800',
      'fcstValue': '20',
      'nx': 59,
      'ny': 127},
     {'baseDate': '20210420',
      'baseTime': '1400',
      'category': 'PTY',
      'fcstDate': '20210420',
      'fcstTime': '1800',
      'fcstValue': '0',
      'nx': 59,
      'ny': 127},
     {'baseDate': '20210420',
      'baseTime': '1400',
      'category': 'R06',
      'fcstDate': '20210420',
      'fcstTime': '1800',
      'fcstValue': '0',
      'nx': 59,
      'ny': 127},
     {'baseDate': '20210420',
      'baseTime': '1400',
      'category': 'REH',
      'fcstDate': '20210420',
      'fcstTime': '1800',
      'fcstValue': '45',
      'nx': 59,
      'ny': 127},
     {'baseDate': '20210420',
      'baseTime': '1400',
      'category': 'S06',
      'fcstD

In [6]:
# 판다스이용 테이블 자료로 변환 
res = pd.DataFrame(data['response']['body']['items']['item'])
res

baseDate baseTime category  fcstDate fcstTime fcstValue  nx   ny
0  20210420     1400      POP  20210420     1800        20  59  127
1  20210420     1400      PTY  20210420     1800         0  59  127
2  20210420     1400      R06  20210420     1800         0  59  127
3  20210420     1400      REH  20210420     1800        45  59  127
4  20210420     1400      S06  20210420     1800         0  59  127
5  20210420     1400      SKY  20210420     1800         3  59  127
6  20210420     1400      T3H  20210420     1800        18  59  127
7  20210420     1400      UUU  20210420     1800       1.8  59  127
8  20210420     1400      VEC  20210420     1800       239  59  127
9  20210420     1400      VVV  20210420     1800       1.1  59  127

In [7]:
# 출력 결과를 JSON으로 가공 출력
result = {}
for index, row in res.iterrows():
#     print(str(row['category']))
    
    print("\t", w_items[str(row['category'])][0], " : ", get_real_value(row['category'], row['fcstValue']), end = "")
    result[w_items[str(row['category'])][0]] =  get_real_value(row['category'], row['fcstValue'])    
    if w_items[str(row['category'])][1]  != "":
        result[w_items[str(row['category'])][0]] =  get_real_value(row['category'], row['fcstValue']) + w_items[str(row['category'])][1] 
        print(w_items[str(row['category'])][1])
    else:
        print()


	 강수확률  :  20%
	 강수형태  :  없음
	 6시간 강수량  :  0mm
	 습도  :  45%
	 6시간 신적설  :  0cm
	 하늘상태  :  구름조금
	 3시간 기온  :  18°C
	 풍속(동서성분)  :  1.8m/s
	 풍향  :  WSW
	 풍속(남북성분)  :  1.1m/s


In [8]:
# Serializing json   
result_json_object = json.dumps(result, indent = 4, ensure_ascii = False)  
print(result_json_object) 

{
    "강수확률": "20%",
    "강수형태": "없음",
    "6시간 강수량": "0mm",
    "습도": "45%",
    "6시간 신적설": "0cm",
    "하늘상태": "구름조금",
    "3시간 기온": "18°C",
    "풍속(동서성분)": "1.8m/s",
    "풍향": "WSW",
    "풍속(남북성분)": "1.1m/s"
}
